# Preparing the data

## Importing libraries, downloading the model

In [ ]:
import pandas
import sklearn
import numpy
import spacy
import string
import sys
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

print(pandas.__version__)
print(sklearn.__version__)
print(numpy.__version__)

1.5.3
1.2.2
1.22.4


In [ ]:
# Small Russian model:
# !python -m spacy download ru_core_news_sm
# nlp = spacy.load('ru_core_news_sm')

# Large Russian model:
!python -m spacy download ru_core_news_lg
nlp = spacy.load('ru_core_news_lg')

## Making lists and doc objects from csv files

In [ ]:
# Let's load the training data from a csv file
train_set = pandas.read_csv('./train_data.csv', encoding='utf-8')
# train_set

In [ ]:
test_set = pandas.read_csv('./test_data.csv', encoding='utf-8')
# test_set

In [ ]:
train_sentences = train_set['text'].to_list()
train_authors = train_set['author'].to_list()

test_sentences = test_set['text'].to_list()
test_authors = test_set['author'].to_list()

print(len(train_authors), len(test_authors))

10000 1000


In [ ]:
train_doc_sentences = nlp.pipe(train_sentences)
test_doc_sentences = nlp.pipe(test_sentences)

## Normalizing the text, replacing "..." with "…"

In [ ]:
def replace_ellipsis(sentences):
  updated_sentences = []
  for sentence in sentences:
    updated_sentence = sentence.replace("...", "…")
    updated_sentences.append(updated_sentence)
  return updated_sentences

train_sentences = replace_ellipsis(train_sentences)
test_sentences = replace_ellipsis(test_sentences)

## Adding additional punctuation marks into string.punctuation

In [ ]:
new_punctuation_string = "—«»–‹›…" + string.punctuation
print(new_punctuation_string)
print(len(new_punctuation_string))

—«»–‹›…!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
39


In [ ]:
# We are creating a matrix with zero vectors for each review (in training set and test set)
train_features_matrix = numpy.zeros((len(train_sentences), len(new_punctuation_string)))
print(train_features_matrix.shape)

test_features_matrix = numpy.zeros((len(test_sentences), len(new_punctuation_string)))
print(test_features_matrix.shape)

(10000, 39)
(1000, 39)


# Modifying the feature vectors

## Visualisation tests

In [ ]:
counter = 0

for sentence, author in zip(train_sentences, train_authors):
    print('Author:', author)
    print(sentence)
    for char in sentence:
      if char in new_punctuation_string:
        print(char)
        char_id = new_punctuation_string.index(char)
        print(char_id)
    counter +=1
    if counter == 2:
      break
        # sys.exit()

Author: Dostoevsky
Но каково же было мое изумление, когда Наташа с первых же слов остановила меня и сказала, что нечего ее утешать, что она уже пять дней, как знает про это..     – Боже мой!
,
18
,
18
,
18
,
18
.
20
.
20
–
3
!
7
Author: Gogol
— закричали в толпе.. — Давай совет кошевой!
—
0
.
20
.
20
—
0
!
7


Visualisation test 2:

In [ ]:
counter = 0

for sentence, author, feature_vector in zip(train_sentences, train_authors, train_features_matrix):
    print('Author:', author)
    print(sentence)
    for char in sentence:
      if char in new_punctuation_string:
        print(char)
        char_id = new_punctuation_string.index(char)
        print(char_id)
        feature_vector[char_id] = 1
        print(feature_vector.tolist())
        # sys.exit()
    counter +=1
    if counter == 2:
      break

Author: Dostoevsky
Но каково же было мое изумление, когда Наташа с первых же слов остановила меня и сказала, что нечего ее утешать, что она уже пять дней, как знает про это..     – Боже мой!
,
18
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
,
18
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
,
18
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
,
18
[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
.
20


## Writing a function for vector modification

In [ ]:
def modify_feature_vectors(sentences, features_matrix):
  for sentence, feature_vector in zip(sentences, features_matrix):
    for char in sentence:
      if char in new_punctuation_string:
        char_id = new_punctuation_string.index(char)
        feature_vector[char_id] = 1
  return features_matrix

In [ ]:
train_features_matrix = numpy.zeros((len(train_sentences), len(new_punctuation_string)))
train_features_matrix_final = modify_feature_vectors(train_sentences, train_features_matrix)

print(train_features_matrix_final[:5])

[[0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


# Training

In [ ]:
lr_punct = LogisticRegression()

# Train the model on the data, storing the information learned from the dat`a
# Model is learning the relationship between digits (x_train) and labels (y_train)
lr_punct.fit(train_features_matrix_final, train_authors)

print(lr_punct.classes_)
print(lr_punct.get_params())

['Chekhov' 'Dostoevsky' 'Gogol' 'Tolstoy']
{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# Modifying the test set feature vectors

In [ ]:
test_features_matrix = numpy.zeros((len(test_sentences), len(new_punctuation_string)))
test_features_matrix_final = modify_feature_vectors(test_sentences, test_features_matrix)

print(test_features_matrix_final[:5])

[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


# Making predictions

In [ ]:
def predict(i):
    print(test_sentences[i])
    # print the features of the index
    print(test_features_matrix_final[i])
    # print the correct label of the index
    print(test_authors[i])

    print()
    print("Prediction:")
    # print the prediction for the features of this index
    print(lr_punct.predict([test_features_matrix_final[i]]))
    # print the probabilities for each label predictions
    print(lr_punct.predict_proba([test_features_matrix_final[i]]))
    print()

In [ ]:
predict(0)
predict(1)
predict(2)
predict(3)
predict(4)
predict(5)

"Фома Фомич, говорю, разве это возможное дело?
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Dostoevsky

Prediction:
['Dostoevsky']
[[0.01173762 0.44792683 0.13174912 0.40858642]]

Пора бы уже домой.
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Chekhov

Prediction:
['Tolstoy']
[[0.24520035 0.21544688 0.24942938 0.28992339]]

А казаки все до одного прощались, зная, что много будет работы тем и другим, но не повершили, однако ж, тотчас разлучиться, а повершили дождаться темной ночной поры, чтоб не дать неприятелю увидеть убыль в казацком войске.
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Gogol

Prediction:
['Tolstoy']
[[0.20652203 0.25246681 0.24582304 0.29518812]]

Вдруг слезы градом у обоих из глаз, дрогнули руки.
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [ ]:
test_predictions_punct = lr_punct.predict(test_features_matrix_final)

for p, r in zip(test_predictions_punct[:10], test_authors[:10]):
    if p == r:
        result = "Correct"
    else:
        result = "Incorrect"
    print(f"{p} ({result}:{r})")

Dostoevsky(Correct:Dostoevsky)
Tolstoy(Incorrect:Chekhov)
Tolstoy(Incorrect:Gogol)
Tolstoy(Incorrect:Dostoevsky)
Tolstoy(Incorrect:Gogol)
Tolstoy(Incorrect:Dostoevsky)
Tolstoy(Incorrect:Dostoevsky)
Chekhov(Correct:Chekhov)
Dostoevsky(Correct:Dostoevsky)
Gogol(Incorrect:Dostoevsky)


# Saving the model

In [ ]:
# Save to file in the current working directory
pkl_filename = "logreg_punkt.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(lr_punct, file)

# Evaluating the model

## Dummy Model

In [ ]:
dummy_predictions = ['Dostoevsky'] * len(test_sentences)
print(len(dummy_predictions))

# Calculate the accuracy of these "dummy predictions"
acc_dummy = accuracy_score(test_authors, dummy_predictions)
print(f'The accuracy is: {acc_dummy}')
print()

print(classification_report(test_authors, dummy_predictions))

1000
The accuracy is: 0.25

              precision    recall  f1-score   support

     Chekhov       0.00      0.00      0.00       250
  Dostoevsky       0.25      1.00      0.40       250
       Gogol       0.00      0.00      0.00       250
     Tolstoy       0.00      0.00      0.00       250

    accuracy                           0.25      1000
   macro avg       0.06      0.25      0.10      1000
weighted avg       0.06      0.25      0.10      1000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Punctuation model

In [ ]:
print('Accuracy:')

acc = accuracy_score(test_authors, test_predictions_punct)
print(acc)
corr_count = accuracy_score(test_authors, test_predictions_punct, normalize=False)
total_count = len(test_authors)

print(f'Total reviews: {str(total_count)}')
print(f'Total correct predictions: {str(corr_count)}')
corr_ratio = corr_count / total_count
print(f'Correct ratio: {str(corr_ratio)}')

Accuracy:
0.371
Total reviews: 1000
Total correct predictions:371
Correct ratio:0.371


In [ ]:
print(classification_report(test_authors, test_predictions_punct))

              precision    recall  f1-score   support

     Chekhov       0.53      0.36      0.43       250
  Dostoevsky       0.35      0.20      0.26       250
       Gogol       0.37      0.28      0.32       250
     Tolstoy       0.32      0.64      0.43       250

    accuracy                           0.37      1000
   macro avg       0.39      0.37      0.36      1000
weighted avg       0.39      0.37      0.36      1000

